In [3]:
import pandas as pd
import numpy as np
import math

In [4]:
# t table
dict1={0.5:[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],0.25:[1.000,0.816,0.765,0.741,0.727,0.718,0.711,0.706,0.703,0.700,0.697,0.695,0.694,0.692,0.691,0.690,0.689,0.688,0.688,0.687,0.686,0.686,0.685,0.685,0.684,0.684,0.684,0.683,0.683,0.683],0.2:[1.376,1.061,0.978,0.941,0.920,0.906,0.896,0.889,0.883,0.879,0.876,0.873,0.870,0.868,0.866,0.865,0.863,0.862,0.861,0.860,0.859,0.858,0.858,0.857,0.856,0.856,0.855,0.855,0.854,0.854],0.15:[1.963,1.386,1.250,1.190,1.156,1.134,1.119,1.108,1.100,1.093,1.088,1.083,1.079,1.076,1.074,1.071,1.069,1.067,1.066,1.064,1.063,1.061,1.060,1.059,1.058,1.058,1.057,1.056,1.055,1.055],0.10:[3.078,1.886,1.638,1.533,1.476,1.440,1.415,1.397,1.383,1.372,1.363,1.356,1.350,1.345,1.341,1.337,1.333,1.330,1.328,1.325,1.323,1.321,1.319,1.318,1.316,1.315,1.314,1.313,1.311,1.310],0.05:[6.314,2.920,2.353,2.132,2.015,1.943,1.895,1.860,1.833,1.812,1.796,1.782,1.771,1.761,1.753,1.746,1.740,1.734,1.729,1.725,1.721,1.717,1.714,1.711,1.708,1.706,1.703,1.701,1.699,1.697], 0.025:[12.71 ,4.303 ,3.182 ,2.776 ,2.571 ,2.447 ,2.365 ,2.306 ,2.262 ,2.228,2.201,2.179,2.160,2.145,2.131,2.120,2.110,2.101,2.093,2.086,2.080,2.074,2.069,2.064,2.060,2.056,2.052,2.048,2.045,2.042],0.01:[31.82,6.965,4.541,3.747,3.365,3.143,2.998,2.896,2.821,2.764,2.718,2.681,2.650,2.624,2.602,2.583,2.567,2.552,2.539,2.528,2.518,2.508,2.500,2.492,2.485,2.479,2.473,2.467,2.462,2.457],0.005:[63.66,9.925,5.841,4.604,4.032,3.707,3.499,3.355,3.250,3.169,3.106,3.055,3.012,2.977,2.947,2.921,2.898,2.878,2.861,2.845,2.831,2.819,2.807,2.797,2.787,2.779,2.771,2.763,2.756,2.750],0.001:[318.31,22.327,10.215,7.173 ,5.893 ,5.208 ,4.785 ,4.501 ,4.297 ,4.144 ,4.025 ,3.930 ,3.852 ,3.787 ,3.733 ,3.686 ,3.646 ,3.610 ,3.579 ,3.552 ,3.527 ,3.505 ,3.485 ,3.467 ,3.450 ,3.435 ,3.421 ,3.408 ,3.396 ,3.385 ],0.0005:[636.62,31.599,12.924,8.610,6.869,5.959,5.408,5.041,4.781,4.587,4.437,4.318,4.221,4.140,4.073,4.015,3.965,3.922,3.883,3.850,3.819,3.792,3.768,3.745,3.725,3.707,3.690,3.674,3.659,3.646]}
df_t=pd.DataFrame(dict1, index=list(range(1,31)))

In [5]:
def estimate(csv=None,col=0,alpha=0.1,pop_std=None,sample_mean=None,sample_std=None,sample_size=None):
    '''
    Finds the point and interval estimate using the appropriate statistic(z or t) 
    
    Provide either a csv with sample data or the  required sample statistics
    
    Parameters:    csv: str, optional
                        Name of the csv file with the sample data
                   col: int, default 0
                        Incase of many columns in the csv. Select the column number for the analysis
                    
                   alpha: float, default 0.1
                        Significance level. alpha=1-confidence interval.
                        Values - 0.1, 0.05 (for sample size>30 - z statistic)
                                 1.0, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.02, 0.01, 0.002, 0.001 (for sample size<30 - t statistic)                                
                        
                   pop_std: float, optional
                        Standard deviation for the population. If available use this
                        for z-statistic or else the sample standard deviation.
                        
                   sample_mean: float,optional(Provide in case of no csv)
                        Mean of the sample. 
                   sample_std: float,optional(Provide in case of no csv)
                        Standard deviation of the sample.
                   sample_size: int,optional(Provide in case of no csv)
                        Size of the sample.
    '''
    z={0.05:1.645, 0.025:1.96}
    flag=0
    if csv!=None:
        try:
            df=pd.read_csv(csv)
        except:
            print("Enter valid csv file name")
        else:
            #Selecting the required data from the csv
            df1=df.select_dtypes(include=np.number) # remove non numeric columns
            col_name=df1.columns.values[col]  #column name
            print(col_name," data")
            df2=df1.iloc[:,col]   #get the column values            
            
            sample_size=df2.count()
            sample_mean=df2.mean()
            sample_std=df2.std()
            
    elif sample_mean==None or sample_std==None or sample_size==None :
        print("Provide the necessary data")
        return
    
    # Calculations for IE    
    try:
        if sample_size>30:
            flag=1
            std=pop_std if pop_std!=None else sample_std  # if population sd is provided use that for the z statistic
            E=z[alpha/2]*std/math.sqrt(sample_size) 
        else:
            std=sample_std
            E=df_t[alpha/2][sample_size-1]*std/math.sqrt(sample_size)
    except:
        print("Check the documentation for the available alpha value")
    else:
        interval_estimate = str(sample_mean-E)+ ' to ' + str(sample_mean+E)
        txt="z" if flag==1 else "t"
        print(f'This is a {txt} distribution with')
        print('Point estimate = ', sample_mean)
        print('Interval estimate = ', interval_estimate)
        

In [6]:
estimate('sample(z-stat).csv')

Salary  data
This is a z distribution with
Point estimate =  180921.19589041095
Interval estimate =  177501.06763248483 to 184341.32414833707


In [7]:
estimate('sample(z-stat).csv', pop_std=2000) # if population sd is available

Salary  data
This is a z distribution with
Point estimate =  180921.19589041095
Interval estimate =  180835.09265514486 to 181007.29912567703


In [8]:
estimate('sample(t-stat).csv')

Salary  data
This is a t distribution with
Point estimate =  190435.29411764705
Interval estimate =  161178.29201318906 to 219692.29622210504


In [9]:
estimate('HouseData.csv',col=1)

MSSubClass  data
This is a z distribution with
Point estimate =  56.897260273972606
Interval estimate =  55.0761522658876 to 58.71836828205761


In [10]:
estimate(sample_mean=18000,sample_std=200,sample_size=15)

This is a t distribution with
Point estimate =  18000
Interval estimate =  17909.06235103105 to 18090.93764896895


In [11]:
estimate(sample_mean=18000,sample_std=200,sample_size=45, alpha=0.1)

This is a z distribution with
Point estimate =  18000
Interval estimate =  17950.955575693504 to 18049.044424306496


In [12]:
estimate() # cant leave it empty

Provide the necessary data


In [13]:
estimate(sample_mean=18000,sample_size=45, alpha=0.1) # sample sd missing

Provide the necessary data
